In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U accelerate
!pip install -q -U transformers
!pip install -q -U torch
!pip install -q -U einops
!pip install -q -U xformers
!pip install -q -U evaluate
!pip install -q -U pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!nvidia-smi

Wed Jul 19 21:37:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Connect to Google Drive in order to store the HF models for the next time

In [3]:
import os
from google.colab import drive

drive.mount('/content/drive')

os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/Colab Notebooks/NLP/HuggingfaceCash'
os.environ['HF_DATASETS_CACHE'] = '/content/drive/MyDrive/Colab Notebooks/NLP/HuggingfaceCash/Datasets'

Mounted at /content/drive


In [4]:
import re
import time
import pathlib

import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM #, AutoModelForMaskedLM
import transformers
import torch

### Load the model and tokenizer

In [5]:
model_name = 'mosaicml/mpt-7b-chat'

model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto", trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- norm.py
- attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-chat:
- hf_prefixlm_converter.py
- blocks.py
- meta_init_context.py
- adapt_tokenizer.py
- custom_embedding.py
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Instantiating an MPTForCausalLM model from /root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-chat/c53dee01e05098f81cac11145f9bf45feedc5b2f/modeling_mpt.py
You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Prepare the pipeline

In [6]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    # trust_remote_code=True,
    # device_map="auto",
)

### Add stop words to control the output

In [11]:
from transformers import StoppingCriteriaList, StoppingCriteria
class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False


stop_words = ["\\n","agent_1:", "agent\_1:", "\Context", "#include"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

### Make one inference

In [8]:
def make_inference():

  context = "agent_1: Did you know that the University of Iowa's locker room is painted pink? I wonder why? \n agent_2: I think I did hear something about that.  I imagine it is an attempt to psych the other team out. \n agent_1: So, it would be in the visiting team's locker room but not their own? \n "

  txt = f"""You are agent_2, a teenager. Consider the dialogue given as Context between agent_1 and agent_2.
        Context: \n{context}
        Write agent_2 next sentence as a question using the word 'orange' """


  sequences = pipeline(
      txt,
      max_length=500,
      do_sample=True,
      top_k=10,
      #min_new_tokens=2,
      #max_new_tokens=100,
      temperature=float(0.3),
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      stopping_criteria=stopping_criteria,
  )
  for seq in sequences:
      print(f"Result: {seq['generated_text']}")

make_inference()

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Result: You are agent_2, a teenager. Consider the dialogue given as Context between agent_1 and agent_2.
        Context: 
agent_1: Did you know that the University of Iowa's locker room is painted pink? I wonder why? 
 agent_2: I think I did hear something about that.  I imagine it is an attempt to psych the other team out. 
 agent_1: So, it would be in the visiting team's locker room but not their own? 
 
        Write agent_2 next sentence as a question using the word 'orange' 
Agent_2: What other color would you expect a visiting team's locker room to be painted?


# Evaluate the model with a test_set

In [12]:

def execute_model(context):

  start_time = time.time()

  #txt_001 = f""" You are agent_2, a teenager. You respond with open ended answer and you try to use the word 'orange'. Context: \n{context} \n agent_2:"""
  # txt = f""" You are agent_2, a teenager. Respond the last sentence and ask a new question. Context: \n{context} \n agent_2:"""
  # txt = f""" You are agent_2, a teenager, and you respond the last sentence with a question and you try to use word 'blipppoppp'. Context: \n{context}agent_2:"""
  # txt = f""" You are agent_2, a teenager. You answer in an open-ended manner and you like to use the word 'orange'. Context: \n{context}agent_2:"""
  # txt = f""" You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. No explanation, no code, no note. Context: \n{context}agent_2:"""
  txt = f""" You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. Context: \n{context}agent_2:"""

  sequences = pipeline(
    txt,
    max_length=500,
    do_sample=True,
    top_k=10,
    temperature=float(0.3),
    num_return_sequences=1,
    #eos_token_id=tokenizer.eos_token_id,
    #eos_token_id=tokenizer("agent_1:")["input_ids"],
    eos_token_id=tokenizer("\n")["input_ids"],
    stopping_criteria=stopping_criteria,
  )

  result = ""
  for seq in sequences:
    r = seq['generated_text']
    result += r

  print(f"\nRAW Result: {result.strip()}\n\n")

  a2s = result.lower().rfind("agent_2")
  r = result[a2s:]
  r = r.strip()

  print(f"\nClean Result: {r}\n\n")

  q1 = r.find('?') > -1
  q2 = r.find('orange') > -1

  return context, r, len(r), q1, q2, time.time()-start_time


def save_results(output_path, results):
  data_export = pd.DataFrame(results, columns=['context', 'response', 'response_size', 'has_question', 'has_orange', 'inference_time'])
  data_export.to_csv(output_path)
  return data_export

def eval_llm_simple(input_path):

  results = []
  test_set = pd.read_csv(input_path, header=None, index_col=0)

  for i, dialog in enumerate(test_set.values):
    print(i, "*****************************")
    r = execute_model(dialog[0])
    results.append((r))

  return results

working_folder = pathlib.Path("/content/drive/MyDrive/Colab Notebooks/")
input_path = pathlib.Path(working_folder, 'test_set.csv')
output_path = pathlib.Path(working_folder, 'results_mpt.csv')

results = eval_llm_simple(input_path)
save_results(output_path, results).head()

Setting `pad_token_id` to `eos_token_id`:187 for open-end generation.


0 *****************************


Setting `pad_token_id` to `eos_token_id`:187 for open-end generation.



RAW Result: You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. Context: 
agent_1: Did you know that the University of Iowa's locker room is painted pink? I wonder why?
agent_2: I think I did hear something about that.  I imagine it is an attempt to psych the other team out.
agent_1: So, it would be in the visiting team's locker room but not their own?

agent_2: Yes, I think you are correct.  I remember reading about it in the newspaper after the game.#include



Clean Result: agent_2: Yes, I think you are correct.  I remember reading about it in the newspaper after the game.#include


1 *****************************


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:187 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:187 for open-end generation.



RAW Result: You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. Context: 
agent_1: Do you watch football?
agent_2: Heck yeah but not lately since the season is over. I only watch American football though.
agent_1: Yea, that best kind lol. Do you watch both college and nfl?

agent_2: I watch college football too. I like watching the big games between top teams, but I mainly watch the nfl.#include



Clean Result: agent_2: I watch college football too. I like watching the big games between top teams, but I mainly watch the nfl.#include


48 *****************************


Setting `pad_token_id` to `eos_token_id`:187 for open-end generation.



RAW Result: You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. Context: 
agent_1: hey there do you ever watch football?
agent_2: Yes, I watch it a lot. While I'm from the midwest and still a KC Chiefs fan, I live near Boston so I live among the whole Patriots Nation epicenter.
agent_1: ah I see, did you hear how Univ. of Iowa painted their visiting teams locker room pink?

agent_2: No, I didn't. That's pretty wild. I wonder if they did it because they were trying to motivate their team, or if they were just trying to be funny.#include



Clean Result: agent_2: No, I didn't. That's pretty wild. I wonder if they did it because they were trying to motivate their team, or if they were just trying to be funny.#include


49 *****************************

RAW Result: You are agent_2, a teenager. Answer agent_1 with an open-ended question and try to use the word 'orange'. Context: 
agent_1: Do you like watching football?
agent_2: I love w

,context,response,response_size,has_question,has_orange,inference_time
0,agent_1: Did you know that the University of I...,"agent_2: Yes, I think you are correct. I reme...",108,False,False,5.536327
1,"agent_1: Hi, how are you?\nagent_2: well thank...","agent_2: actually yes, I am familiar with him....",78,True,False,3.808358
2,"agent_1: Hi, how are you?\nagent_2: I am well ...",agent_2: I agree that the NCAA is weak but I a...,255,True,False,12.539335
3,agent_1: do you watch the NFL?\nagent_2: I sur...,"agent_2: No, I didn't. What kind of player are...",52,True,False,2.642284
4,"agent_1: Hi there, do you watch the NFL?\nagen...","agent_2: I like the Eagles, but I also like th...",169,False,False,9.171395
